In [1]:
import os
import sys
import glob

import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras.backend.tensorflow_backend import set_session

import matplotlib.pyplot as plt

import nibabel as nib

Using TensorFlow backend.


In [2]:
def load_nii(path):
    nii = nib.load(path)
    vol = nii.get_data().astype(np.float32)
    return np.squeeze(vol)

In [3]:
run_dirs = glob.glob('../runs/*/predict')
run_dirs = sorted(run_dirs)

print('found {} runs'.format(len(run_dirs)), '\n')

for i, run in enumerate(run_dirs):
    print(i, run, '\n')

found 7 runs 

0 ../runs/gan_20190428_1448_gpu=7_bs=4_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=50.0_vr=0.5_is=5_cs=5_rf=10_rs=5_sw=True_glw=[1_100_500_10_0]_clw=[1_1_10]/predict 

1 ../runs/gan_20190430_1720_gpu=7_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=50.0_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_glw=[1_100_500_10_0_0]_clw=[1_1_10]_tag=beefier_decoder_more_summaries_batchnorm/predict 

2 ../runs/gan_20190503_1139_gpu=2_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25.0_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_reg=True_clf=False_glw=[1.0_500.0_10.0_250.0_0.0_0.0_50.0]_clw=[1.0_1.0_10.0]_tag=new_arch/predict 

3 ../runs/gan_20190505_1942_gpu=7_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_reg=True_clf=False_glw=[1.0_100.0_10.0_100.0_0.0_0.0_25.0]_clw=[1_1_10]_tag=train_s1_eval_s2/predict 

4 ../runs/gan_20190505_2247_gpu=6_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5

In [7]:
# set run dir
run_dir = run_dirs[5]

pre_dirs = glob.glob(os.path.join(glob.escape(run_dir), '*', '*'))
pre_dirs = sorted(pre_dirs)

print('found {} pre_dirs'.format(len(pre_dirs)))

for i, pre_dir in enumerate(pre_dirs):
    #print(i, os.path.basename(pre_dir))
    print(i, pre_dir.split('/')[-2] + '/' + pre_dir.split('/')[-1])

found 2 pre_dirs
0 test/gen_100_04
1 test/gen_500_04


In [8]:
# generator config, run_dir, etc...
pre_dir = pre_dirs[1]

csv_path = os.path.join(pre_dir, 'meta.csv')
csv = pd.read_csv(csv_path)

print('found meta.csv: {}'.format(csv.shape))

found meta.csv: (271, 43)


In [13]:
# classifier
clf_dir = '../../dx/runs/clf_20190501_0021_gpu=4_bs=64_lr=0.0001_b1=0.9_b2=0.999_ep=0.1_bn=True_mp=True_lk=0.0_ls=[4_2_8_1_8_2_32_1_32_2_128_1_128_1_256_1]_tag=fast_drop/'
model_name = 'clf_1000'
model_file = os.path.join(clf_dir, model_name+'.h5')

if os.path.isfile(model_file):
    print('found model file')
else:
    print('model file not found!')

found model file


In [10]:
# gpu config
gpu_id = 3

if gpu_id is not None:                                                     
    gpu = '/gpu:' + str(gpu_id)                                            
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)                       
    config = tf.ConfigProto()                                              
    config.gpu_options.allow_growth = True                                 
    config.allow_soft_placement = True                                     
    set_session(tf.Session(config=config))                                 
else:                                                                      
    gpu = '/cpu:0'

In [14]:
print('starting predict')

with tf.device(gpu):
    
    # load classifier model
    classifier = keras.models.load_model(model_file)
    
    for i, (_, row) in enumerate(csv.iterrows()):
        
        if i % 10 == 0:
            print('step', i)
        
        xr = load_nii(row['img_path_0'])[None, ..., None]
        yr = load_nii(row['img_path_1'])[None, ..., None]
        yf = load_nii(row['img_path_yf'])[None, ..., None]
        
        batch = np.concatenate([xr, yr, yf], axis=0)
        
        pred = classifier.predict([batch])
        
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_xr'] = pred[0, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yr'] = pred[1, 1]
        csv.loc[csv.img_id_0 == row['img_id_0'], 'pAD_yf'] = pred[2, 1]
        
print('finished predict')

csv_out_path = os.path.join(pre_dir, model_name+'.csv')

print('writing csv to {}'.format(csv_out_path))

csv.to_csv(csv_out_path, index=False)

starting predict
step 0
step 10
step 20
step 30
step 40
step 50
step 60
step 70
step 80
step 90
step 100
step 110
step 120
step 130
step 140
step 150
step 160
step 170
step 180
step 190
step 200
step 210
step 220
step 230
step 240
step 250
step 260
step 270
finished predict
writing csv to ../runs/gan_20190506_1227_gpu=1_bs=16_cl=8_lr=0.0001_b1=0.0_b2=0.9_ep=0.1_pl=25_lk=0.2_bn=True_vr=0.5_is=6_cs=5_rf=10_rs=5_sw=True_reg=False_clf=True_glw=[1.0_100.0_10.0_100.0_0.0_0.0_25.0]_clw=[1_1_10]_tag=clf_lower_clf_loss/predict/test/gen_500_04/clf_1000.csv


In [16]:
clf_csvs = glob.glob(os.path.join(glob.escape(pre_dir), '[!meta]*.csv'))
clf_csvs = sorted(clf_csvs)

for i, clf_csv in enumerate(clf_csvs):
    print(i, os.path.basename(clf_csv))

0 clf_0500.csv
1 clf_1000.csv


In [31]:
csv_path = clf_csvs[1]
csv = pd.read_csv(csv_path)
print(csv.shape)

(271, 46)


In [32]:
print(csv[['pAD_xr', 'pAD_yr', 'pAD_yf', 'pat_dx_1']].mean())
csv[['pAD_xr', 'pAD_yr', 'pAD_yf', 'pat_dx_1']].head()*100

pAD_xr      0.236336
pAD_yr      0.409310
pAD_yf      0.765941
pat_dx_1    2.361624
dtype: float64


,pAD_xr,pAD_yr,pAD_yf,pat_dx_1
0,0.000011,0.005754,56.429178,200.0
1,74.181962,98.772484,99.989498,200.0
2,0.008689,7.760223,99.957758,200.0
3,0.214881,39.731318,11.490502,200.0
4,4.751689,16.493386,35.216990,200.0


In [33]:
def get_metric(csv_in, col, t, metric):

    true_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 3)).sum()
    true_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 2)).sum()
    false_pos = ((csv_in[col] > t) & (csv_in.pat_dx_1 == 2)).sum()
    false_neg = ((csv_in[col] <= t) & (csv_in.pat_dx_1 == 3)).sum()
    
    true = true_pos + true_neg
    false = false_pos + false_neg

    if metric == 'f1':
        prec = true_pos / (true_pos + false_pos)
        reca = true_pos / (true_pos + false_neg)

        f1_score = 2 * (prec * reca) / (prec + reca)
        m = f1_score
        
    elif metric == 'acc':
        acc_p = true_pos / (true_pos + false_neg)
        acc_s = true_neg / (true_neg + false_pos)

        balanced_acc = (acc_p + acc_s) / 2
        m = balanced_acc
    
    return (round(m, 3), true, false)

In [34]:
def get_best_threshold(csv_in, col, metric):
    l = []
    
    for t in np.arange(0, 1, 0.01):
        m = get_metric(csv_in, col, t, metric)
        #l.append((*m, t))
        l.append((m[0], -t))
        
    return -(max(l)[1])

In [35]:
def get_split_metric(csv_in, col, cv_split, metric):
    
    csv_split = csv_in[csv_in.cv_split == cv_split]
    csv_other = csv_in[csv_in.cv_split != cv_split]
    
    t = get_best_threshold(csv_other, col, metric)
    
    m = get_metric(csv_split, col, t, metric)
    print('split {}: {}, t={}'.format(cv_split, m, t))
    
    return m

In [36]:
def get_cv_metric(csv_in, col, metric):
    l = [get_split_metric(csv_in, col, s, metric)[0] for s in range(5)]
    return np.array(l).mean()

In [37]:
# create 5 folds
csv = csv.sample(frac=1).reset_index(drop=True)

pMCI = csv[csv.pat_dx_1 == 3].img_id_0.values
sMCI = csv[csv.pat_dx_1 == 2].img_id_0.values

p_bins = np.linspace(0, len(pMCI), 6).astype(int)[1:-1]
s_bins = np.linspace(0, len(sMCI), 6).astype(int)[1:-1]

p_splits = np.split(pMCI, p_bins)
s_splits = np.split(sMCI, s_bins)

In [38]:
# write folds to csv
for i, (p_split, s_split) in enumerate(zip(p_splits, s_splits)):
    csv.loc[csv.img_id_0.isin(p_split), 'cv_split'] = i
    csv.loc[csv.img_id_0.isin(s_split), 'cv_split'] = i

In [39]:
get_cv_metric(csv, 'pAD_xr', 'f1')

split 0: (0.683, 40, 13), t=0.03
split 1: (0.682, 41, 14), t=0.03
split 2: (0.533, 32, 21), t=0.06
split 3: (0.667, 38, 17), t=0.03
split 4: (0.571, 37, 18), t=0.06


0.6272

In [40]:
get_cv_metric(csv, 'pAD_yr', 'f1')

split 0: (0.711, 40, 13), t=0.18
split 1: (0.735, 42, 13), t=0.18
split 2: (0.653, 36, 17), t=0.1
split 3: (0.72, 41, 14), t=0.19
split 4: (0.591, 37, 18), t=0.2


0.682

In [41]:
get_cv_metric(csv, 'pAD_yf', 'f1')

split 0: (0.5, 29, 24), t=0.96
split 1: (0.525, 26, 29), t=0.85
split 2: (0.56, 31, 22), t=0.96
split 3: (0.593, 33, 22), t=0.96
split 4: (0.615, 35, 20), t=0.96


0.5586

In [42]:
get_cv_metric(csv, 'pAD_xr', 'acc')

split 0: (0.719, 41, 12), t=0.17
split 1: (0.732, 43, 12), t=0.17
split 2: (0.619, 35, 18), t=0.17
split 3: (0.729, 39, 16), t=0.06
split 4: (0.657, 37, 18), t=0.06


0.6912

In [43]:
get_cv_metric(csv, 'pAD_yr', 'acc')

split 0: (0.774, 40, 13), t=0.18
split 1: (0.793, 42, 13), t=0.19
split 2: (0.715, 36, 17), t=0.1
split 3: (0.779, 41, 14), t=0.18
split 4: (0.668, 37, 18), t=0.2


0.7458

In [44]:
get_cv_metric(csv, 'pAD_yf', 'acc')

split 0: (0.566, 29, 24), t=0.96
split 1: (0.639, 35, 20), t=0.98
split 2: (0.546, 30, 23), t=0.99
split 3: (0.643, 33, 22), t=0.96
split 4: (0.629, 35, 20), t=0.98


0.6046